# agora a passar para csv 

In [ ]:
import requests
import re
import json
import csv
from datetime import datetime
import math

# import extra , tava  a testar 
# from bs4 import BeautifulSoup as bs
# from datetime import datetime

API_KEY = "9fa7ce317d6e85c90d92244adb9146c6"
BASE_URL = "https://api.itjobs.pt/job/list.json"

def fetch_skills(skills, start_date, end_date):
    """Busca vagas que requerem habilidades específicas dentro de um intervalo de datas."""
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    skills = skills.split(',') if isinstance(skills, str) else skills
    URL = f'{BASE_URL}?api_key={API_KEY}'
    all_skills = []

    try:
        initial_request = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        initial_request.raise_for_status()
        total_pages = min(math.ceil(initial_request.json().get('total', 0) / 12), 100)

        for page in range(1, total_pages + 1):
            response = requests.get(URL, params={'limit': 12, 'page': page}, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
            response.raise_for_status()
            
            for item in response.json().get('results', []):
                try:
                    published_date = datetime.strptime(item['publishedAt'], '%Y-%m-%d %H:%M:%S')
                except ValueError:
                    continue
                
                if start_date <= published_date <= end_date and all(re.search(r'\b' + re.escape(skill) + r'\b', item.get('body', ''), re.IGNORECASE) for skill in skills):
                    all_skills.append({
                        'id': item['id'],
                        'title': item.get('title', ''),
                        'company_name': item.get('company', {}).get('name', ''),
                        'publishedAt': item['publishedAt'],
                        'wage': item.get('wage', ''),
                        'address': item.get('company', {}).get('address', '')
                    })

        return all_skills

    except requests.RequestException as e:
        print("Erro na requisição:", e)
        return []


# teste 

# fetch_skill("Python,Django", "2024-10-01", "2024-10-31")




# aqui e para tornar isto em csv


def save_csv(data, filename):
    fieldnames = ['id', 'title', 'company_name', 'publishedAt', 'wage', 'address']
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    print(f"Dados salvos em {filename}")

def show_skills(skills, start_date, end_date, output_file="skills.csv"):
    results = fetch_skills(skills, start_date, end_date)
    print(json.dumps(results, indent=4, ensure_ascii=False))
    save_to_csv(results, output_file)

# Exemplo de uso
show_skills("Python,Django", "2024-10-01", "2024-10-31", output_file="skills.csv")
